**SA433 &#x25aa; Data Wrangling and Visualization &#x25aa; Fall 2024**

# Lesson 13. Filtering Observations in Pandas

## Overview

- Over the next several lessons, we'll learn how to perform 5 fundamental data wrangling tasks with tabular data:

    1. Filter rows by their values
    2. Sort the rows
    3. Select and drop columns
    4. Create new columns with functions of existing columns
    5. Collapse many values down to a summary


- These 5 tasks combined will allow us to solve a majority of our data wrangling challenges

## In this lesson...

- How do we filter, or pick rows, based on their values?


- An aside: how are missing values represented in Pandas?

<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## The nycflights13 dataset

- Let's begin by importing Pandas

In [1]:
import pandas as pd

- In the same folder as this notebook is a file `data/nycflights13_flights.csv.zip`
    - This is a ZIP file containing a single CSV file
    - ZIP files not only package multiple files together, they also compress their contents to save disk space


- In this case, there's no need to extract the CSV file: `pd.read_csv()` can read ZIPped CSV files directly:

In [2]:
df = pd.read_csv('data/nycflights13_flights.csv.zip')

- This dataset, based on data from the [US Bureau of Transportation Statistics](https://www.transtats.bts.gov/DatabaseInfo.asp?DB_ID=120&Link=0), contains information about all 336,776 flights that departed from New York City in 2013


- Each row corresponds to one flight, and contains the following data:

| Column | Description |
| :- | :- |
| `year`, `month`, `day` | Date of departure |
| `dep_time`, `arr_time` | Actual departure and arrival times (format HHMM or HMM), local timezone |
| `sched_dep_time`, `sched_arr_time` | Scheduled departure and arrival times (format HHMM or HMM), local timezone |
| `dep_delay`, `arr_delay` | Departure and arrival delays, in minutes; negative times represent early departures/arrivals |
| `carrier` | Two letter carrier abbreviation |
| `flight` | Flight number |
| `tailnum` | Plane tail number |
| `origin`, `dest` | Origin and destination |
| `air_time` | Amount of time spent in the air, in minutes |
| `distance` | Distance between airports, in miles |
| `hour`, `minute` | Time of scheduled departure broken into hour and minutes |
| `time_hour` | Scheduled date and hour of the flight |



- Let's take a quick look:

In [3]:
df.head()

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,1545,N14228,EWR,IAH,227.0,1400,5,15,2013-01-01T10:00:00Z
1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,1714,N24211,LGA,IAH,227.0,1416,5,29,2013-01-01T10:00:00Z
2,2013,1,1,542.0,540,2.0,923.0,850,33.0,AA,1141,N619AA,JFK,MIA,160.0,1089,5,40,2013-01-01T10:00:00Z
3,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,B6,725,N804JB,JFK,BQN,183.0,1576,5,45,2013-01-01T10:00:00Z
4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,DL,461,N668DN,LGA,ATL,116.0,762,6,0,2013-01-01T11:00:00Z


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336776 entries, 0 to 336775
Data columns (total 19 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   year            336776 non-null  int64  
 1   month           336776 non-null  int64  
 2   day             336776 non-null  int64  
 3   dep_time        328521 non-null  float64
 4   sched_dep_time  336776 non-null  int64  
 5   dep_delay       328521 non-null  float64
 6   arr_time        328063 non-null  float64
 7   sched_arr_time  336776 non-null  int64  
 8   arr_delay       327346 non-null  float64
 9   carrier         336776 non-null  object 
 10  flight          336776 non-null  int64  
 11  tailnum         334264 non-null  object 
 12  origin          336776 non-null  object 
 13  dest            336776 non-null  object 
 14  air_time        327346 non-null  float64
 15  distance        336776 non-null  int64  
 16  hour            336776 non-null  int64  
 17  minute    

<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## The query method

- We can filter observations based on their values using the `.query()` method of a DataFrame


- `.query()` takes a Python expression as a *string*, in which the variable names refer to the columns of the DataFrame


- For example, we can select all flights operated by United Airlines (UA) with:

In [5]:
# Solution
df.query('carrier == "UA"')

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,1545,N14228,EWR,IAH,227.0,1400,5,15,2013-01-01T10:00:00Z
1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,1714,N24211,LGA,IAH,227.0,1416,5,29,2013-01-01T10:00:00Z
5,2013,1,1,554.0,558,-4.0,740.0,728,12.0,UA,1696,N39463,EWR,ORD,150.0,719,5,58,2013-01-01T10:00:00Z
12,2013,1,1,558.0,600,-2.0,924.0,917,7.0,UA,194,N29129,JFK,LAX,345.0,2475,6,0,2013-01-01T11:00:00Z
13,2013,1,1,558.0,600,-2.0,923.0,937,-14.0,UA,1124,N53441,EWR,SFO,361.0,2565,6,0,2013-01-01T11:00:00Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336723,2013,9,30,2052.0,2059,-7.0,2315.0,2358,-43.0,UA,1416,N68453,EWR,MCO,123.0,937,20,59,2013-10-01T00:00:00Z
336728,2013,9,30,2055.0,2059,-4.0,2158.0,2211,-13.0,UA,1066,N16701,EWR,BOS,40.0,200,20,59,2013-10-01T00:00:00Z
336737,2013,9,30,2105.0,2106,-1.0,2329.0,2354,-25.0,UA,475,N477UA,EWR,IAH,175.0,1400,21,6,2013-10-01T01:00:00Z
336755,2013,9,30,2149.0,2156,-7.0,2245.0,2308,-23.0,UA,523,N813UA,EWR,BOS,37.0,200,21,56,2013-10-01T01:00:00Z


- Note that `df.query()` outputs another DataFrame: it doesn't remove the undesired observations from `df`


- If you want to save the resulting DataFrame, you can assign it to another variable:

In [6]:
# Solution
ua_df = df.query('carrier == "UA"')

### Comparison operators

- The standard Python comparison operators (i.e., `==`, `!=`, `>`, `<`, `>=`, `<=`) all work with `.query()`


- For example, we can select all flights that spent more than 400 minutes in the air like this:

In [7]:
# Solution
df.query('air_time > 400')

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
162,2013,1,1,857.0,900,-3.0,1516.0,1530,-14.0,HA,51,N380HA,JFK,HNL,659.0,4983,9,0,2013-01-01T14:00:00Z
379,2013,1,1,1344.0,1344,0.0,2005.0,1944,21.0,UA,15,N76065,EWR,HNL,656.0,4963,13,44,2013-01-01T18:00:00Z
1073,2013,1,2,909.0,900,9.0,1525.0,1530,-5.0,HA,51,N380HA,JFK,HNL,638.0,4983,9,0,2013-01-02T14:00:00Z
1293,2013,1,2,1344.0,1344,0.0,1940.0,1944,-4.0,UA,15,N77066,EWR,HNL,634.0,4963,13,44,2013-01-02T18:00:00Z
2018,2013,1,3,914.0,900,14.0,1504.0,1530,-26.0,HA,51,N380HA,JFK,HNL,616.0,4983,9,0,2013-01-03T14:00:00Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334536,2013,9,28,1327.0,1329,-2.0,1710.0,1813,-63.0,UA,15,N66057,EWR,HNL,562.0,4963,13,29,2013-09-28T17:00:00Z
335095,2013,9,29,957.0,1000,-3.0,1405.0,1445,-40.0,HA,51,N384HA,JFK,HNL,580.0,4983,10,0,2013-09-29T14:00:00Z
335301,2013,9,29,1420.0,1329,51.0,1817.0,1813,4.0,UA,15,N66057,EWR,HNL,569.0,4963,13,29,2013-09-29T17:00:00Z
336081,2013,9,30,959.0,1000,-1.0,1438.0,1445,-7.0,HA,51,N392HA,JFK,HNL,603.0,4983,10,0,2013-09-30T14:00:00Z


### Logical operators

- The standard Python logical operators (i.e., `and`, `or`, `not`) also work with `.query()`


- For example, we can select all flights on January 1 like this:

In [8]:
# Solution
df.query('(month == 1) and (day == 1)')

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,1545,N14228,EWR,IAH,227.0,1400,5,15,2013-01-01T10:00:00Z
1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,1714,N24211,LGA,IAH,227.0,1416,5,29,2013-01-01T10:00:00Z
2,2013,1,1,542.0,540,2.0,923.0,850,33.0,AA,1141,N619AA,JFK,MIA,160.0,1089,5,40,2013-01-01T10:00:00Z
3,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,B6,725,N804JB,JFK,BQN,183.0,1576,5,45,2013-01-01T10:00:00Z
4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,DL,461,N668DN,LGA,ATL,116.0,762,6,0,2013-01-01T11:00:00Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
837,2013,1,1,2356.0,2359,-3.0,425.0,437,-12.0,B6,727,N588JB,JFK,BQN,186.0,1576,23,59,2013-01-02T04:00:00Z
838,2013,1,1,NaN,1630,NaN,NaN,1815,NaN,EV,4308,N18120,EWR,RDU,NaN,416,16,30,2013-01-01T21:00:00Z
839,2013,1,1,NaN,1935,NaN,NaN,2240,NaN,AA,791,N3EHAA,LGA,DFW,NaN,1389,19,35,2013-01-02T00:00:00Z
840,2013,1,1,NaN,1500,NaN,NaN,1825,NaN,AA,1925,N3EVAA,LGA,MIA,NaN,1096,15,0,2013-01-01T20:00:00Z


### Membership operators

- The standard Python membership operators (i.e., `in`, `not in`) work with `.query()` as well


- For example, we can select all flights headed to Atlanta (ATL), Chicago O'Hare (ORD), and Los Angeles (LAX) like this:

In [9]:
# Solution
df.query('dest in ["ATL", "ORD", "LAX"]')

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,DL,461,N668DN,LGA,ATL,116.0,762,6,0,2013-01-01T11:00:00Z
5,2013,1,1,554.0,558,-4.0,740.0,728,12.0,UA,1696,N39463,EWR,ORD,150.0,719,5,58,2013-01-01T10:00:00Z
9,2013,1,1,558.0,600,-2.0,753.0,745,8.0,AA,301,N3ALAA,LGA,ORD,138.0,733,6,0,2013-01-01T11:00:00Z
12,2013,1,1,558.0,600,-2.0,924.0,917,7.0,UA,194,N29129,JFK,LAX,345.0,2475,6,0,2013-01-01T11:00:00Z
18,2013,1,1,600.0,600,0.0,837.0,825,12.0,MQ,4650,N542MQ,LGA,ATL,134.0,762,6,0,2013-01-01T11:00:00Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336709,2013,9,30,2032.0,2045,-13.0,2147.0,2225,-38.0,AA,371,N434AA,LGA,ORD,105.0,733,20,45,2013-10-01T00:00:00Z
336721,2013,9,30,2051.0,2054,-3.0,2348.0,2359,-11.0,B6,523,N605JB,JFK,LAX,304.0,2475,20,54,2013-10-01T00:00:00Z
336724,2013,9,30,2053.0,1815,158.0,2310.0,2054,136.0,EV,5292,N600QX,EWR,ATL,91.0,746,18,15,2013-09-30T22:00:00Z
336744,2013,9,30,2121.0,2100,21.0,2349.0,14,-25.0,DL,2363,N193DN,JFK,LAX,296.0,2475,21,0,2013-10-01T01:00:00Z


### Using Python variables

- We can refer to variables in the Python environment by prefixing them with `@`


- For example, we can rewrite the query above as follows:

In [10]:
# Solution
airports = ['ATL', 'ORD', 'LAX']
df.query('dest in @airports')

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,DL,461,N668DN,LGA,ATL,116.0,762,6,0,2013-01-01T11:00:00Z
5,2013,1,1,554.0,558,-4.0,740.0,728,12.0,UA,1696,N39463,EWR,ORD,150.0,719,5,58,2013-01-01T10:00:00Z
9,2013,1,1,558.0,600,-2.0,753.0,745,8.0,AA,301,N3ALAA,LGA,ORD,138.0,733,6,0,2013-01-01T11:00:00Z
12,2013,1,1,558.0,600,-2.0,924.0,917,7.0,UA,194,N29129,JFK,LAX,345.0,2475,6,0,2013-01-01T11:00:00Z
18,2013,1,1,600.0,600,0.0,837.0,825,12.0,MQ,4650,N542MQ,LGA,ATL,134.0,762,6,0,2013-01-01T11:00:00Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336709,2013,9,30,2032.0,2045,-13.0,2147.0,2225,-38.0,AA,371,N434AA,LGA,ORD,105.0,733,20,45,2013-10-01T00:00:00Z
336721,2013,9,30,2051.0,2054,-3.0,2348.0,2359,-11.0,B6,523,N605JB,JFK,LAX,304.0,2475,20,54,2013-10-01T00:00:00Z
336724,2013,9,30,2053.0,1815,158.0,2310.0,2054,136.0,EV,5292,N600QX,EWR,ATL,91.0,746,18,15,2013-09-30T22:00:00Z
336744,2013,9,30,2121.0,2100,21.0,2349.0,14,-25.0,DL,2363,N193DN,JFK,LAX,296.0,2475,21,0,2013-10-01T01:00:00Z


### Column names with spaces

- If one of the columns has a name with a space, we can refer to it by surrounding its name in backticks `` ` ``


- For example, if we wanted to filter for rows whose values of columns `A` and `B B` are not equal, we could write:

    ```python
    df.query('A != `B B`')
    ```

<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## Missing values

- Let's take a look at flight AA 133 on January 2:

In [11]:
# Solution
df.query('(carrier == "AA") and (flight == 133) and (month == 1) and (day == 2)')

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
1782,2013,1,2,NaN,1545,NaN,NaN,1910,NaN,AA,133,NaN,JFK,LAX,NaN,2475,15,45,2013-01-02T20:00:00Z


- As we can see, many of the variables for this particular flight have a value of `NaN` (short for "Not a Number")


- By default, `NaN` is how Pandas represents **missing values**: values that, for one reason or another, do not exist


- We can pick rows whose columns have missing values with `.isna()`


- For example, we can filter for the rows with missing tail numbers like this:

In [12]:
# Solution
df.query('tailnum.isna()')

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
1782,2013,1,2,NaN,1545,NaN,NaN,1910,NaN,AA,133,NaN,JFK,LAX,NaN,2475,15,45,2013-01-02T20:00:00Z
1784,2013,1,2,NaN,1601,NaN,NaN,1735,NaN,UA,623,NaN,EWR,ORD,NaN,719,16,1,2013-01-02T21:00:00Z
2697,2013,1,3,NaN,857,NaN,NaN,1209,NaN,UA,714,NaN,EWR,MIA,NaN,1085,8,57,2013-01-03T13:00:00Z
2698,2013,1,3,NaN,645,NaN,NaN,952,NaN,UA,719,NaN,EWR,DFW,NaN,1372,6,45,2013-01-03T11:00:00Z
3608,2013,1,4,NaN,845,NaN,NaN,1015,NaN,9E,3405,NaN,JFK,DCA,NaN,213,8,45,2013-01-04T13:00:00Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334186,2013,9,27,NaN,1600,NaN,NaN,1739,NaN,UA,269,NaN,LGA,ORD,NaN,733,16,0,2013-09-27T20:00:00Z
334868,2013,9,28,NaN,1635,NaN,NaN,1827,NaN,US,581,NaN,EWR,CLT,NaN,529,16,35,2013-09-28T20:00:00Z
335782,2013,9,29,NaN,700,NaN,NaN,833,NaN,UA,331,NaN,LGA,ORD,NaN,733,7,0,2013-09-29T11:00:00Z
336771,2013,9,30,NaN,1455,NaN,NaN,1634,NaN,9E,3393,NaN,JFK,DCA,NaN,213,14,55,2013-09-30T18:00:00Z


- *Note.* If you have the `numexpr` package installed in your Python environment, to use `.isna()` in `.query()`, you need to force `.query()` to use the `python` parser with the keyword argument `engine='python'`


- Perhaps we want to filter for the rows that *do not* have missing tail numbers instead, which we can do like this:

In [13]:
# Solution
df.query('not (tailnum.isna())')

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,1545,N14228,EWR,IAH,227.0,1400,5,15,2013-01-01T10:00:00Z
1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,1714,N24211,LGA,IAH,227.0,1416,5,29,2013-01-01T10:00:00Z
2,2013,1,1,542.0,540,2.0,923.0,850,33.0,AA,1141,N619AA,JFK,MIA,160.0,1089,5,40,2013-01-01T10:00:00Z
3,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,B6,725,N804JB,JFK,BQN,183.0,1576,5,45,2013-01-01T10:00:00Z
4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,DL,461,N668DN,LGA,ATL,116.0,762,6,0,2013-01-01T11:00:00Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336769,2013,9,30,2349.0,2359,-10.0,325.0,350,-25.0,B6,745,N516JB,JFK,PSE,196.0,1617,23,59,2013-10-01T03:00:00Z
336770,2013,9,30,NaN,1842,NaN,NaN,2019,NaN,EV,5274,N740EV,LGA,BNA,NaN,764,18,42,2013-09-30T22:00:00Z
336773,2013,9,30,NaN,1210,NaN,NaN,1330,NaN,MQ,3461,N535MQ,LGA,BNA,NaN,764,12,10,2013-09-30T16:00:00Z
336774,2013,9,30,NaN,1159,NaN,NaN,1344,NaN,MQ,3572,N511MQ,LGA,CLE,NaN,419,11,59,2013-09-30T15:00:00Z


- It turns out that in this case, we can also use the `.notna()` method, like this:

In [14]:
# Solution
df.query('tailnum.notna()')

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,1545,N14228,EWR,IAH,227.0,1400,5,15,2013-01-01T10:00:00Z
1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,1714,N24211,LGA,IAH,227.0,1416,5,29,2013-01-01T10:00:00Z
2,2013,1,1,542.0,540,2.0,923.0,850,33.0,AA,1141,N619AA,JFK,MIA,160.0,1089,5,40,2013-01-01T10:00:00Z
3,2013,1,1,544.0,545,-1.0,1004.0,1022,-18.0,B6,725,N804JB,JFK,BQN,183.0,1576,5,45,2013-01-01T10:00:00Z
4,2013,1,1,554.0,600,-6.0,812.0,837,-25.0,DL,461,N668DN,LGA,ATL,116.0,762,6,0,2013-01-01T11:00:00Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336769,2013,9,30,2349.0,2359,-10.0,325.0,350,-25.0,B6,745,N516JB,JFK,PSE,196.0,1617,23,59,2013-10-01T03:00:00Z
336770,2013,9,30,NaN,1842,NaN,NaN,2019,NaN,EV,5274,N740EV,LGA,BNA,NaN,764,18,42,2013-09-30T22:00:00Z
336773,2013,9,30,NaN,1210,NaN,NaN,1330,NaN,MQ,3461,N535MQ,LGA,BNA,NaN,764,12,10,2013-09-30T16:00:00Z
336774,2013,9,30,NaN,1159,NaN,NaN,1344,NaN,MQ,3572,N511MQ,LGA,CLE,NaN,419,11,59,2013-09-30T15:00:00Z


- We'll learn about ways to deal with missing values in a later lesson

<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## Problems

For the problems below, use the nycflights13 dataset we used in this lesson.

### Problem 1

Find all flights that had an arrival delay of 2 or more hours.

In [15]:
# Solution
df.query('arr_delay >= 120')

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
119,2013,1,1,811.0,630,101.0,1047.0,830,137.0,MQ,4576,N531MQ,LGA,CLT,118.0,544,6,30,2013-01-01T11:00:00Z
151,2013,1,1,848.0,1835,853.0,1001.0,1950,851.0,MQ,3944,N942MQ,JFK,BWI,41.0,184,18,35,2013-01-01T23:00:00Z
218,2013,1,1,957.0,733,144.0,1056.0,853,123.0,UA,856,N534UA,EWR,BOS,37.0,200,7,33,2013-01-01T12:00:00Z
268,2013,1,1,1114.0,900,134.0,1447.0,1222,145.0,UA,1086,N76502,LGA,IAH,248.0,1416,9,0,2013-01-01T14:00:00Z
447,2013,1,1,1505.0,1310,115.0,1638.0,1431,127.0,EV,4497,N17984,EWR,RIC,63.0,277,13,10,2013-01-01T18:00:00Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336579,2013,9,30,1823.0,1545,158.0,1934.0,1733,121.0,9E,3459,N916XJ,JFK,BNA,95.0,765,15,45,2013-09-30T19:00:00Z
336668,2013,9,30,1951.0,1649,182.0,2157.0,1903,174.0,EV,4294,N13988,EWR,SAV,95.0,708,16,49,2013-09-30T20:00:00Z
336724,2013,9,30,2053.0,1815,158.0,2310.0,2054,136.0,EV,5292,N600QX,EWR,ATL,91.0,746,18,15,2013-09-30T22:00:00Z
336757,2013,9,30,2159.0,1845,194.0,2344.0,2030,194.0,9E,3320,N906XJ,JFK,BUF,50.0,301,18,45,2013-09-30T22:00:00Z


### Problem 2

Find all flights that flew to Houston (IAH or HOU).

In [16]:
# Solution
df.query('dest in ["IAH", "HOU"]')

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
0,2013,1,1,517.0,515,2.0,830.0,819,11.0,UA,1545,N14228,EWR,IAH,227.0,1400,5,15,2013-01-01T10:00:00Z
1,2013,1,1,533.0,529,4.0,850.0,830,20.0,UA,1714,N24211,LGA,IAH,227.0,1416,5,29,2013-01-01T10:00:00Z
32,2013,1,1,623.0,627,-4.0,933.0,932,1.0,UA,496,N459UA,LGA,IAH,229.0,1416,6,27,2013-01-01T11:00:00Z
81,2013,1,1,728.0,732,-4.0,1041.0,1038,3.0,UA,473,N488UA,LGA,IAH,238.0,1416,7,32,2013-01-01T12:00:00Z
89,2013,1,1,739.0,739,0.0,1104.0,1038,26.0,UA,1479,N37408,EWR,IAH,249.0,1400,7,39,2013-01-01T12:00:00Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336524,2013,9,30,1729.0,1720,9.0,2001.0,2010,-9.0,UA,652,N455UA,EWR,IAH,173.0,1400,17,20,2013-09-30T21:00:00Z
336527,2013,9,30,1735.0,1715,20.0,2010.0,2005,5.0,WN,2067,N296WN,EWR,HOU,188.0,1411,17,15,2013-09-30T21:00:00Z
336618,2013,9,30,1859.0,1859,0.0,2134.0,2159,-25.0,UA,1128,N14731,LGA,IAH,180.0,1416,18,59,2013-09-30T22:00:00Z
336694,2013,9,30,2015.0,2015,0.0,2244.0,2307,-23.0,UA,1545,N17730,EWR,IAH,174.0,1400,20,15,2013-10-01T00:00:00Z


### Problem 3

Find all flights that were operated by Southwest (WN), Frontier (F9), or Alaska (AS).
If you didn't already in Problem 2, try using the `in` membership operator.

In [17]:
# Solution
df.query('carrier in ["WN", "F9", "AS"]')

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
39,2013,1,1,629.0,630,-1.0,721.0,740,-19.0,WN,4646,N273WN,LGA,BWI,40.0,185,6,30,2013-01-01T11:00:00Z
78,2013,1,1,724.0,725,-1.0,1020.0,1030,-10.0,AS,11,N594AS,EWR,SEA,338.0,2402,7,25,2013-01-01T12:00:00Z
102,2013,1,1,754.0,755,-1.0,1103.0,1030,33.0,WN,733,N789SW,LGA,DEN,279.0,1620,7,55,2013-01-01T12:00:00Z
145,2013,1,1,833.0,835,-2.0,1134.0,1102,32.0,F9,835,N203FR,LGA,DEN,257.0,1620,8,35,2013-01-01T13:00:00Z
156,2013,1,1,852.0,855,-3.0,1046.0,1020,26.0,WN,3848,N777QC,EWR,MDW,146.0,711,8,55,2013-01-01T13:00:00Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336586,2013,9,30,1829.0,1835,-6.0,2153.0,2148,5.0,AS,5,N528AS,EWR,SEA,341.0,2402,18,35,2013-09-30T22:00:00Z
336615,2013,9,30,1857.0,1830,27.0,2107.0,2100,7.0,WN,2698,N232WN,EWR,MSY,149.0,1167,18,30,2013-09-30T22:00:00Z
336636,2013,9,30,1915.0,1900,15.0,2025.0,2015,10.0,WN,490,N719SW,LGA,MKE,104.0,738,19,0,2013-09-30T23:00:00Z
336710,2013,9,30,2033.0,2030,3.0,2143.0,2150,-7.0,WN,2520,N286WN,EWR,MDW,97.0,711,20,30,2013-10-01T00:00:00Z


### Problem 4

Find all flights that departed in the summer (June, July, August, September).

In [18]:
# Solution
df.query('(month >= 6) and (month <= 9)')

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
222207,2013,6,1,2.0,2359,3.0,341.0,350,-9.0,B6,739,N618JB,JFK,PSE,200.0,1617,23,59,2013-06-02T03:00:00Z
222208,2013,6,1,451.0,500,-9.0,624.0,640,-16.0,US,1431,N538UW,EWR,CLT,75.0,529,5,0,2013-06-01T09:00:00Z
222209,2013,6,1,506.0,515,-9.0,715.0,800,-45.0,UA,1686,N35407,EWR,IAH,172.0,1400,5,15,2013-06-01T09:00:00Z
222210,2013,6,1,534.0,545,-11.0,800.0,829,-29.0,UA,1451,N27724,LGA,IAH,178.0,1416,5,45,2013-06-01T09:00:00Z
222211,2013,6,1,538.0,545,-7.0,925.0,922,3.0,B6,725,N806JB,JFK,BQN,203.0,1576,5,45,2013-06-01T09:00:00Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336771,2013,9,30,NaN,1455,NaN,NaN,1634,NaN,9E,3393,NaN,JFK,DCA,NaN,213,14,55,2013-09-30T18:00:00Z
336772,2013,9,30,NaN,2200,NaN,NaN,2312,NaN,9E,3525,NaN,LGA,SYR,NaN,198,22,0,2013-10-01T02:00:00Z
336773,2013,9,30,NaN,1210,NaN,NaN,1330,NaN,MQ,3461,N535MQ,LGA,BNA,NaN,764,12,10,2013-09-30T16:00:00Z
336774,2013,9,30,NaN,1159,NaN,NaN,1344,NaN,MQ,3572,N511MQ,LGA,CLE,NaN,419,11,59,2013-09-30T15:00:00Z


### Problem 5

Find all flights that arrived more than 2 hours late, but didn't leave late.

In [19]:
# Solution
df.query('(dep_delay <= 0) and (arr_delay > 120)')

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
22911,2013,1,27,1419.0,1420,-1.0,1754.0,1550,124.0,MQ,3728,N1EAMQ,EWR,ORD,135.0,719,14,20,2013-01-27T19:00:00Z
33011,2013,10,7,1350.0,1350,0.0,1736.0,1526,130.0,EV,5181,N611QX,LGA,MSN,117.0,812,13,50,2013-10-07T17:00:00Z
33019,2013,10,7,1357.0,1359,-2.0,1858.0,1654,124.0,AA,1151,N3CMAA,LGA,DFW,192.0,1389,13,59,2013-10-07T17:00:00Z
41075,2013,10,16,657.0,700,-3.0,1258.0,1056,122.0,B6,3,N703JB,JFK,SJU,225.0,1598,7,0,2013-10-16T11:00:00Z
55985,2013,11,1,658.0,700,-2.0,1329.0,1015,194.0,VX,399,N629VA,JFK,LAX,336.0,2475,7,0,2013-11-01T11:00:00Z
152766,2013,3,18,1844.0,1847,-3.0,39.0,2219,140.0,UA,389,N560UA,JFK,SFO,386.0,2586,18,47,2013-03-18T22:00:00Z
180893,2013,4,17,1635.0,1640,-5.0,2049.0,1845,124.0,MQ,4540,N721MQ,LGA,DTW,130.0,502,16,40,2013-04-17T20:00:00Z
181270,2013,4,18,558.0,600,-2.0,1149.0,850,179.0,AA,707,N3EXAA,LGA,DFW,234.0,1389,6,0,2013-04-18T10:00:00Z
181327,2013,4,18,655.0,700,-5.0,1213.0,950,143.0,AA,2083,N565AA,EWR,DFW,230.0,1372,7,0,2013-04-18T11:00:00Z
213693,2013,5,22,1827.0,1830,-3.0,2217.0,2010,127.0,MQ,4674,N518MQ,LGA,CLE,90.0,419,18,30,2013-05-22T22:00:00Z


### Problem 6

Find all flights that departed in the morning (before 1200) *or* in the evening (after 1800) on July 19. 

In [20]:
# Solution
df.query('((month == 7) and (day == 19)) and ((dep_time <= 1200) or (dep_time >= 1800))')

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
267400,2013,7,19,24.0,1940,284.0,201.0,2150,251.0,EV,4543,N17138,EWR,DSM,141.0,1017,19,40,2013-07-19T23:00:00Z
267401,2013,7,19,25.0,2359,26.0,358.0,340,18.0,B6,839,N598JB,JFK,BQN,193.0,1576,23,59,2013-07-20T03:00:00Z
267402,2013,7,19,30.0,2050,220.0,407.0,42,205.0,UA,1071,N37465,EWR,BQN,198.0,1585,20,50,2013-07-20T00:00:00Z
267403,2013,7,19,37.0,2155,162.0,303.0,43,140.0,B6,425,N519JB,JFK,TPA,126.0,1005,21,55,2013-07-20T01:00:00Z
267404,2013,7,19,55.0,2359,56.0,443.0,350,53.0,B6,745,N591JB,JFK,PSE,205.0,1617,23,59,2013-07-20T03:00:00Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
268379,2013,7,19,2333.0,2030,183.0,46.0,2202,164.0,9E,4079,N826AY,JFK,BWI,35.0,184,20,30,2013-07-20T00:00:00Z
268380,2013,7,19,2333.0,2245,48.0,110.0,3,67.0,B6,486,N323JB,JFK,ROC,49.0,264,22,45,2013-07-20T02:00:00Z
268381,2013,7,19,2337.0,2028,189.0,131.0,2247,164.0,B6,135,N653JB,JFK,PHX,265.0,2153,20,28,2013-07-20T00:00:00Z
268382,2013,7,19,2342.0,2305,37.0,48.0,13,35.0,B6,718,N292JB,JFK,BOS,40.0,187,23,5,2013-07-20T03:00:00Z


### Problem 7

Find all flights that were delayed by at least an hour, but made up over 30 minutes in flight.

*Hint.* You can use arithmetic operators (`+`, `-`, `*`, `/`) in combination with comparison operators directly in `.query()`.

In [21]:
# Solution
df.query('(dep_delay >= 60) and (dep_delay - arr_delay >= 30)')

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
593,2013,1,1,1716.0,1545,91.0,2140.0,2039,61.0,B6,703,N651JB,JFK,SJU,183.0,1598,15,45,2013-01-01T20:00:00Z
815,2013,1,1,2205.0,1720,285.0,46.0,2040,246.0,AA,1999,N5DNAA,EWR,MIA,146.0,1085,17,20,2013-01-01T22:00:00Z
832,2013,1,1,2326.0,2130,116.0,131.0,18,73.0,B6,199,N594JB,JFK,LAS,290.0,2248,21,30,2013-01-02T02:00:00Z
2286,2013,1,3,1503.0,1221,162.0,1803.0,1555,128.0,UA,551,N835UA,EWR,SFO,320.0,2565,12,21,2013-01-03T17:00:00Z
2495,2013,1,3,1821.0,1530,171.0,2131.0,1910,141.0,AA,85,N357AA,JFK,SFO,328.0,2586,15,30,2013-01-03T20:00:00Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336572,2013,9,30,1818.0,1715,63.0,2001.0,1940,21.0,9E,3310,N934XJ,JFK,MCI,142.0,1113,17,15,2013-09-30T21:00:00Z
336579,2013,9,30,1823.0,1545,158.0,1934.0,1733,121.0,9E,3459,N916XJ,JFK,BNA,95.0,765,15,45,2013-09-30T19:00:00Z
336674,2013,9,30,1956.0,1825,91.0,2208.0,2121,47.0,DL,1576,N3772H,JFK,SAN,292.0,2446,18,25,2013-09-30T22:00:00Z
336704,2013,9,30,2028.0,1910,78.0,2255.0,2215,40.0,AA,21,N338AA,JFK,LAX,294.0,2475,19,10,2013-09-30T23:00:00Z


### Problem 8

How many flights have a missing departure time? What other variables are missing for these flights? What might these rows represent?

In [22]:
# Solution
df.query('dep_time.isna()')

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,flight,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
838,2013,1,1,NaN,1630,NaN,NaN,1815,NaN,EV,4308,N18120,EWR,RDU,NaN,416,16,30,2013-01-01T21:00:00Z
839,2013,1,1,NaN,1935,NaN,NaN,2240,NaN,AA,791,N3EHAA,LGA,DFW,NaN,1389,19,35,2013-01-02T00:00:00Z
840,2013,1,1,NaN,1500,NaN,NaN,1825,NaN,AA,1925,N3EVAA,LGA,MIA,NaN,1096,15,0,2013-01-01T20:00:00Z
841,2013,1,1,NaN,600,NaN,NaN,901,NaN,B6,125,N618JB,JFK,FLL,NaN,1069,6,0,2013-01-01T11:00:00Z
1777,2013,1,2,NaN,1540,NaN,NaN,1747,NaN,EV,4352,N10575,EWR,CVG,NaN,569,15,40,2013-01-02T20:00:00Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336771,2013,9,30,NaN,1455,NaN,NaN,1634,NaN,9E,3393,NaN,JFK,DCA,NaN,213,14,55,2013-09-30T18:00:00Z
336772,2013,9,30,NaN,2200,NaN,NaN,2312,NaN,9E,3525,NaN,LGA,SYR,NaN,198,22,0,2013-10-01T02:00:00Z
336773,2013,9,30,NaN,1210,NaN,NaN,1330,NaN,MQ,3461,N535MQ,LGA,BNA,NaN,764,12,10,2013-09-30T16:00:00Z
336774,2013,9,30,NaN,1159,NaN,NaN,1344,NaN,MQ,3572,N511MQ,LGA,CLE,NaN,419,11,59,2013-09-30T15:00:00Z


*Write your answer here. Double-click to edit.*

*Solution.* For flights with missing `dep_time`, it looks like `arr_time`, `air_time`, `dep_delay` and `arr_delay` are also missing. It seems like these flights never flew, so perhaps these flights were canceled.

<hr style="border-top: 2px solid gray; margin-top: 1px; margin-bottom: 1px"></hr>

## Notes and sources

- From the [Pandas User Guide](https://pandas.pydata.org/docs/user_guide/index.html):
    - [The query method](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#the-query-method)


- Lesson and problems inspired by Chapter 5 of [R for Data Science](https://r4ds.had.co.nz/)